In [5]:
import pandas as pd

The following lines displays floats with two decimal places and expands the limits for the displayed number of rows and columns.

In [3]:
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

Reading sample csv file (and setting the column names):

In [13]:
df = pd.read_csv('BNC2_sample.csv',
                names=['Code', 'Date', 'Open', 'High', 'Low',
                      'Close', 'Volume', 'VWAP', 'TWAP'])
df.head()

,Code,Date,Open,High,Low,Close,Volume,VWAP,TWAP
0,GWA_BTC,2014-04-01,467.28,488.62,467.28,479.56,"74,776.48",482.76,482.82
1,GWA_BTC,2014-04-02,479.20,494.30,431.32,437.08,"114,052.96",460.19,465.93
2,GWA_BTC,2014-04-03,437.33,449.74,414.41,445.60,"91,415.08",432.29,433.28
3,GWA_BTC,2014-04-04,445.18,456.10,429.16,449.81,"51,147.27",443.46,443.93
4,GWA_BTC,2014-04-05,450.08,464.09,445.16,461.70,"28,449.19",452.53,452.95


Inspecting for equivalency in granularity:

In [6]:
print(df.Code.unique())

['GWA_BTC' 'GWA_ETH' 'GWA_LTC' 'GWA_XLM' 'GWA_XRP' 'MWA_BTC_CNY'
 'MWA_BTC_EUR' 'MWA_BTC_GBP' 'MWA_BTC_JPY' 'MWA_BTC_USD' 'MWA_ETH_CNY'
 'MWA_ETH_EUR' 'MWA_ETH_GBP' 'MWA_ETH_JPY' 'MWA_ETH_USD' 'MWA_LTC_CNY'
 'MWA_LTC_EUR' 'MWA_LTC_GBP' 'MWA_LTC_JPY' 'MWA_LTC_USD' 'MWA_XLM_CNY'
 'MWA_XLM_EUR' 'MWA_XLM_USD' 'MWA_XRP_CNY' 'MWA_XRP_EUR' 'MWA_XRP_GBP'
 'MWA_XRP_JPY' 'MWA_XRP_USD']


SInce global weighted average (GWA) is an aggregation of market weighted average (MWA), we only need to keep GWA.

In [14]:
print('Before:', len(df))
gwa_codes = [code for code in df.Code.unique() if 'GWA_' in code]
df = df[df.Code.isin(gwa_codes)]
print('After:', len(df))

Before: 31761
After: 6309


Pivoting the dataset:

In [15]:
pivoted_df = df.pivot(index='Date', columns='Code', values='VWAP')
pivoted_df.tail()

Code,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
Date,,,,,
2018-01-19,"11,826.36","1,068.45",195.00,0.51,1.82
2018-01-20,"13,062.68","1,158.71",207.58,0.52,1.75
2018-01-21,"12,326.23","1,108.90",197.36,0.48,1.55
2018-01-22,"11,397.52","1,038.21",184.92,0.47,1.43
2018-01-23,"10,921.00",992.05,176.95,0.47,1.42


Suppose we want to calculate the weekly returns for the past month:

In [16]:
delta_7 = pivoted_df/pivoted_df.shift(7)-1.0
delta_7.tail()

Code,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
Date,,,,,
2018-01-19,-0.18,-0.17,-0.18,-0.21,-0.22
2018-01-20,-0.13,-0.19,-0.18,-0.23,-0.29
2018-01-21,-0.15,-0.20,-0.22,-0.22,-0.30
2018-01-22,-0.21,-0.24,-0.24,-0.25,-0.32
2018-01-23,-0.11,-0.12,-0.13,-0.02,-0.04


In [17]:
delta_dict = {}
for offset in [7, 14, 21, 28]:
    delta_dict['delta_{}'.format(offset)] = pivoted_df/pivoted_df.shift(offset)

Melting the delta dataframes:

In [21]:
melted_dfs = []
for key, delta_df in delta_dict.items():
    melted_dfs.append( delta_df.reset_index().melt(id_vars=['Date'], value_name=key))

Creating a new melted dataframe with forward looking 7 day returns:

In [22]:
return_df = pivoted_df.shift(-7)/pivoted_df - 1.0
melted_dfs.append(return_df.reset_index().melt(id_vars=['Date'], value_name='return_7'))

Now we want to merge all the melted dataframes into one base table.

In [23]:
from functools import reduce

In [24]:
base_df = df[['Date', 'Code', 'Volume', 'VWAP']]
feature_dfs = [base_df] + melted_dfs

In [26]:
abt = reduce(lambda left,right: pd.merge(left,right,on=['Date', 'Code']), feature_dfs)
abt.tail(10)

,Date,Code,Volume,VWAP,delta_7,delta_14,delta_21,delta_28,return_7
6299,2018-01-14,GWA_XRP,"912,107,674.18",2.20,0.69,1.02,2.06,2.97,-0.30
6300,2018-01-15,GWA_XRP,"823,491,754.55",2.11,0.71,0.96,1.92,2.84,-0.32
6301,2018-01-16,GWA_XRP,"3,872,977,355.95",1.48,0.54,0.64,1.35,1.85,-0.04
6302,2018-01-17,GWA_XRP,"5,111,390,628.85",1.20,0.53,0.43,0.94,1.58,NaN
6303,2018-01-18,GWA_XRP,"5,156,172,462.44",1.68,0.78,0.49,1.23,1.59,NaN
6304,2018-01-19,GWA_XRP,"2,126,239,927.56",1.82,0.78,0.59,1.02,1.65,NaN
6305,2018-01-20,GWA_XRP,"1,346,913,296.52",1.75,0.71,0.58,0.74,1.53,NaN
6306,2018-01-21,GWA_XRP,"1,886,060,450.81",1.55,0.70,0.49,0.72,1.45,NaN
6307,2018-01-22,GWA_XRP,"1,784,992,299.63",1.43,0.68,0.48,0.65,1.30,NaN
6308,2018-01-23,GWA_XRP,"2,118,335,564.32",1.42,0.96,0.52,0.61,1.29,NaN
